<a href="https://colab.research.google.com/github/AlvaroUriel/idal_ia3/blob/main/Trabajo_Uriel_Aprendizaje_produndo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from  google.colab import drive
 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
train_path = '/content/drive/MyDrive/IA3/Datasets/Food-3/training'
val_path   = '/content/drive/MyDrive/IA3/Datasets/Food-3/validation'
test_path   = '/content/drive/MyDrive/IA3/Datasets/Food-3/evaluation'

In [30]:
import os
import numpy as np

# Keras
from keras.utils import np_utils

In [34]:
#Creamos listas con todos los nombres de los ficheros 
train = [os.path.join(train_path,img) for img in os.listdir(train_path)]
val   = [os.path.join(val_path,img) for img in os.listdir(val_path)]
test   = [os.path.join(val_path,img) for img in os.listdir(val_path)]


len(train),len(val),len(test)

(2923, 301, 301)

In [28]:
train[0:3]

['/content/drive/MyDrive/IA3/Datasets/Food-3/training/1_185.jpg',
 '/content/drive/MyDrive/IA3/Datasets/Food-3/training/2_249.jpg',
 '/content/drive/MyDrive/IA3/Datasets/Food-3/training/1_361.jpg']

In [33]:
#Troceamos por la ultima '/' y por '_' para quedarnos el primer numero y lo pasamos a entero
train_y = [int(img.split("/")[-1].split("_")[0]) for img in train]
val_y = [int(img.split("/")[-1].split("_")[0]) for img in val]
test_y = [int(img.split("/")[-1].split("_")[0]) for img in test]
num_classes = 3

#One hot encoding para tener las etiquetas
y_train = np_utils.to_categorical(train_y, num_classes)
y_val = np_utils.to_categorical(val_y, num_classes)
y_test = np_utils.to_categorical(test_y, num_classes)

In [ ]:
#Vemos lo que obtenemos con el 'one hot'
print(train_y[0:20])
print(y_train[0:20])

In [44]:
print("Datos de entrenamiento en: " + num_classes )


TypeError: ignored